In [64]:
import vectorbt as vbt
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact




In [96]:
# Baixar os dados do índice Ibovespa
df = yf.download('^BVSP', period='5y')

# Calcular o indicador TRIX usando a biblioteca talib
trix = talib.TRIX(df['Close'], timeperiod=4)
trix_ma = pd.Series(trix).rolling(2).mean()

# Criar sinais de entrada e saída
entries = trix > trix_ma
exits = (df['Close'] > (df['Close'].shift() * 1.01))

# Executar o backtest
portfolio = vbt.Portfolio.from_signals(df['Close'], entries, exits, fees=0.001, freq='1D')

# Exibir os resultados do backtest
print(portfolio.stats())

[*********************100%***********************]  1 of 1 completed
Start                                2018-07-20 00:00:00
End                                  2023-07-20 00:00:00
Period                                1240 days 00:00:00
Start Value                                        100.0
End Value                                     117.880999
Total Return [%]                               17.880999
Benchmark Return [%]                           50.288274
Max Gross Exposure [%]                             100.0
Total Fees Paid                                10.913957
Max Drawdown [%]                               37.123877
Max Drawdown Duration                  530 days 00:00:00
Total Trades                                          45
Total Closed Trades                                   44
Total Open Trades                                      1
Open Trade PnL                                  0.253854
Win Rate [%]                                        50.0
Best Trade [%]     

In [97]:
portfolio.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e86f20d7-fb08-4cc3-a80b-152f758cb484',
              'x': array([datetime.datetime(2018, 7, 20, 0, 0),
                          datetime.datetime(2018, 7, 23, 0, 0),
                          datetime.datetime(2018, 7, 24, 0, 0), ...,
                          datetime.datetime(2023, 7, 18, 0, 0),
                          datetime.datetime(2023, 7, 19, 0, 0),
                          datetime.datetime(2023, 7, 20, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 78571.,  77996.,  79155., ..., 117841., 117552., 118083.]),
              'yaxis': 'y'},
             {'customdata': array([[0.00000000e+00, 1.23255851e-03, 9.99000999e-02],
                                   [2.00000000e+00, 1.24891925e-03, 9.53275085e-02],
                             

In [74]:
portfolio.plot_cum_returns()

FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6ae1287e-9efa-481c-8514-2027b5c3267d',
              'x': array([datetime.datetime(2013, 7, 22, 0, 0),
                          datetime.datetime(2013, 7, 23, 0, 0),
                          datetime.datetime(2013, 7, 24, 0, 0), ...,
                          datetime.datetime(2023, 7, 18, 0, 0),
                          datetime.datetime(2023, 7, 19, 0, 0),
                          datetime.datetime(2023, 7, 20, 0, 0)], dtype=object),
              'y': array([1.        , 1.00506444, 0.99588257, ..., 2.4260098 , 2.42006011,
                          2.43099189])},
             {'hoverinfo': 'skip',
              'line': {'color': 'rgba(0, 0, 0, 0)', 'width': 0},
              'opacity': 0,
              'showlegend': False,
              'type': 'scatter',
              'uid': 'c0b5275e-af29-4be2-9403

In [13]:
# Exibir a lista de ordens
print(portfolio.orders.stats())


Start                2020-07-20 00:00:00
End                  2023-07-20 00:00:00
Period                 747 days 00:00:00
Total Records                         34
Total Buy Orders                      17
Total Sell Orders                     17
Min Size                        0.000905
Max Size                        0.001039
Avg Size                        0.000972
Avg Buy Size                    0.000972
Avg Sell Size                   0.000972
Avg Buy Price              110004.352941
Avg Sell Price             110901.235294
Total Fees                      3.643002
Min Fees                        0.097761
Max Fees                        0.116231
Avg Fees                        0.107147
Avg Buy Fees                    0.106738
Avg Sell Fees                   0.107556
dtype: object


In [17]:
# Definir uma função para calcular os sinais de entrada e saída com base nos parâmetros timeperiod e rolling
def trix_signals(close, timeperiod, rolling):
    trix = talib.TRIX(close, timeperiod=timeperiod)
    trix_ma = pd.Series(trix).rolling(rolling).mean()
    entries = trix > trix_ma
    exits = trix < trix_ma
    return entries, exits

# Definir uma grade de parâmetros para testar
param_grid = {
    'timeperiod': range(3, 15),
    'rolling': range(5, 20)
}

# Executar a otimização de grade
portfolio_opt = vbt.Portfolio.from_signals(
    df['Close'],
    *trix_signals(df['Close'], 14, 9),
    fees=0.001,
    freq='1D'
)

# Exibir os resultados da otimização
print(portfolio_opt.stats())




Start                                2020-07-20 00:00:00
End                                  2023-07-20 00:00:00
Period                                 747 days 00:00:00
Start Value                                        100.0
End Value                                     110.268809
Total Return [%]                               10.268809
Benchmark Return [%]                           13.078063
Max Gross Exposure [%]                             100.0
Total Fees Paid                                 3.643002
Max Drawdown [%]                               17.486538
Max Drawdown Duration                  530 days 00:00:00
Total Trades                                          17
Total Closed Trades                                   17
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   41.176471
Best Trade [%]                                 11.787323
Worst Trade [%]                